In [1]:
print("Why hello there")

Why hello there


In [2]:
import cv2
import pytesseract
import os
from os import walk
import pandas as pd
import re
from PIL import Image
from PIL.ExifTags import GPSTAGS, TAGS

In [3]:
dir1 = os.getcwd() + "/ScreenShots"
filenames = next(walk(dir1), (None, None, []))[2]
filenames = sorted(filenames)

In [4]:
# DF FOR TIMESTAMPS ON FILES
timestampList = []
fileNameList = []
for file in filenames:
    try:
        fileNameList.append(file)
        file = dir1 + "/" + file
        image = Image.open(file)
        if image._getexif() == None:
            fileNamstList.pop(-1)
            print(f"{file} contains no exif data.")
        else:
            for tag, value in image._getexif().items():
                tag_name = TAGS.get(tag)
                if tag_name == "DateTimeOriginal":
                    timestampList.append(value)
    except IOError:
        fileNamstList.pop(-1)
        print("File format not supported!")
dfTuples = list(zip(fileNameList, timestampList))    
timeDF = pd.DataFrame(dfTuples, columns=["FileName", "Time"])

,FileName,Time
0,IMG_2279.PNG,2023:06:24 13:17:06
1,IMG_2280.PNG,2023:06:24 13:38:54
2,IMG_2281.PNG,2023:06:24 13:44:10
3,IMG_2283.PNG,2023:06:24 13:59:00
4,IMG_2284.PNG,2023:06:24 14:01:32
5,IMG_2288.PNG,2023:06:24 14:28:18
6,IMG_2289.PNG,2023:06:24 14:32:10
7,IMG_2296.PNG,2023:06:24 15:09:25
8,IMG_2298.PNG,2023:06:24 15:15:23
9,IMG_2301.PNG,2023:06:24 15:42:26


In [5]:
# GOES THROUGH ALL FILES
# READS TEXT FROM IMAGES, SORTS TO DELIVERY OFFERS AND DELIVERY COMPLETIONS
# SAVES TEXT AND FILE NAME APPROPRIATELY
def testFunc(files_to_process_list):
    offerCounter = completeCounter = driveCounter = thirdCounter = 0
    theList = []
    completionList = []
    offerFiles = []
    compFiles = []
    for file in files_to_process_list:
        #file to image
        img = cv2.imread(dir1 + "/" + file)
        #image to text
        data = pytesseract.image_to_string(img)
        #MARKETPLACE DELIVERY OFFER
        if("Guaranteed" in data):
            # The above text should only be found in marketplace delivery offers
            # We want to append the file to the list of offers
            # Limiting text to things after the $ excludes anything read from the map
            # Split the text data into lines and get rid of empty lines
            # If $ cannot be found, split the text into lines and add the entire list to be processed later
            offerFiles.append(file)
            offerCounter += 1
            if("$" in data):
                position = data.index("$")
                textsplit = data[position:].splitlines()
                while ("" in textsplit):
                    textsplit.remove("")
                theList.append(textsplit)
            else:
                data = data.splitlines()
                theList.append(data)
        #DELIVERY COMPLETIONS
        elif("Completed" in data):
            compFiles.append(file)
            img = cv2.imread(dir1 + "/" + file)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
            data = pytesseract.image_to_string(thresh, lang='eng',config='--psm 6')
            # The above filters are applied to improve extraction quality
            completeCounter += 1
            # The below section works as before to exclude anything read from the map
            if("$" in data):
                position = data.index("$")
                textsplit = data[position:].splitlines()
                while("" in textsplit):
                    textsplit.remove("")
                completionList.append(textsplit)
            else:
                print("Error in", file, "$ not found")
                data = data.splitlines()
                completionList.append(data)
        #DRIVE DELIVERY OFFER
        elif("item" in data):
            # The interface has a slightly different format, so split the text into list of lines and process later
            offerFiles.append(file)
            driveCounter += 1
            data = data.splitlines()
            theList.append(data)                   
        else:
            print(file)
            print("ERROR WITH THIS FILE")
            thirdCounter += 1
    return offerCounter, completeCounter, driveCounter, thirdCounter, theList, completionList, offerFiles, compFiles

x = testFunc(filenames)
offers = x[0]
completes = x[1]
drives = x[2]
others = x[3]
offersList = x[4]
completionList = x[5]
offerFiles = x[6]
compFiles = x[7]

In [6]:
betterList = []
badList = []
# For every list of strings in the list of delivery offers
for x in offersList:
    indexA = 0
    indexB = 0
    startVal = endVal = 0
    DriveOrderFlag = 0
    # For every string in the list
    for y in x:
        # If Drive order, add string and break
        if "item" in y:
            betterList.append(x)
            DriveOrderFlag = 1
            break
        # If Marketplace order
        if "Guaranteed" in y:
            startVal = indexA
        if "Customer dropoff" in y:
            endVal = indexB
        indexA +=1
        indexB +=1
        if endVal != 0:
            betterList.append(x[startVal: endVal+1])
            break
    if(endVal == 0 and DriveOrderFlag != 1):
        print("Yikes, adding to awkward list")
        print(indexA, indexB, startVal, endVal)
        badList.append(x)

In [7]:
#print(betterList)
#print(badList)

In [8]:
weirdDFAttempt = pd.DataFrame(betterList)
weirdDFAttempt.fillna("", inplace=True)
# Convert list to DF, drop whitespace
# For every column, for every row
# If "DoorDash pay" found, it is used as an anchor from which other values are found
# Set the column values for column 0, 1, 3, and 4 for ease of later processing
for col in weirdDFAttempt:
    targetRow = targetCol = -1
    for index, value in weirdDFAttempt[col].items():
        if "DoorDash pay" in value:
            targetRow = index
            targetCol = col
            offerVal = weirdDFAttempt[targetCol-2][targetRow]
            mileVal = weirdDFAttempt[targetCol-4][targetRow]
            restVal = weirdDFAttempt[targetCol-6][targetRow]
            weirdDFAttempt.at[targetRow, 0] = offerVal + " Guaranteed"
            weirdDFAttempt.at[targetRow, 1] = mileVal
            weirdDFAttempt.at[targetRow, 3] = "Pickup"
            weirdDFAttempt.at[targetRow, 4] = restVal

In [9]:
# Function to convert $ string to float, 
# I should expand to allow for multiple columns, handle the large DF at end instead of after func def
def dollarToFloat(df):
    index = 0
    for x in df[0]:
        if("$" in x):
            x = x.replace(" ", "")
            x = x.replace("+", "")
            a = x.index("$")
            b = x.index("Guaranteed")
            x = x.replace("$", "")
            the_float = x[a:b-1]
            df.at[index, 0] = float(the_float)
        index +=1
dollarToFloat(weirdDFAttempt)

In [10]:
# This looks for "mi" although technically "yards" can show up as an option, this would not handle that

# Look through columns given, in this case 1 and 2, extract the number and convert to float
# Due to image to text reads the value could be in either column, combine values
# Then for every string in the Series, if "mi" is present, use regex to find the value just before
# Convert that value to float, and place it back in the "Mileage" column
def mi2fl(df, columns):
    df["Mileage"] = df[columns[0]] + " " + df[columns[1]]
    itr = 0
    for x in df["Mileage"]:
        if( "mi" in x):
            #Okay so \S+ is a word and \s+ is whitespace, so this means find 1 word appearing before mi from x
            match = re.search('(\S+\s+){1}(?=mi)', x)
            df.at[itr, "Mileage"] = float(match.group(0))
        else:
            df.at[itr, "Mileage"] = "ERROR"
        itr +=1       
mi2fl(weirdDFAttempt, [1,2])

5.5 
3.8 
4.6 
10.8 
3.4 
4.2 
3.7 
10.5 
1.6 
1.5 
6.8 
2.2 
1.6 
4.0 
3.7 
0.6 
4.1 
2.9 
3.6 
1.7 
3.0 
4.1 
2.9 
8.0 
2.9 
0.5 


In [11]:
weirdDFAttempt = weirdDFAttempt.rename(columns={0: "Dollars"})

In [12]:
# A few things complicate these reads
# The restaurant will always show in [4] OR [5]....so far
# For natural stacks, the second restaurant will appear in [6] or [7]
# This will grab the first restaurant, and set the second to either be "" or the name of the second restaurant
# Or it will save the names as ERROR
def getRestNames(df):
    restNames = []
    rest2Names = []
    for index, row in df.iterrows():
        restName = ""
        rest2 = ""
        if("Pickup" in row[3]):
            restName = row[4].replace("| ", "")
            restNames.append(restName)
            if("Pickup" in row[5]):
                rest2 = row[6]
                rest2Names.append(rest2)
            else:
                rest2Names.append(rest2)
        elif("Pickup" in row[4]):
            restName = row[5].replace("| ", "")
            restNames.append(restName)
            if("Pickup" in row[6]):
                rest2 = row[7]
                rest2Names.append(rest2)
            else:
                rest2Names.append(rest2)
        else:
            print("error")
            restNames.append("ERROR")
            rest2Names.append("ERROR")
    df["RestNames"] = restNames
    df["Rest 2"] = rest2Names 
getRestNames(weirdDFAttempt)

error


In [13]:
# Extract only the useful columns to new DF
offerDF = weirdDFAttempt[["Dollars", "Mileage", "RestNames", "Rest 2"]]

In [14]:
# In this method, we immediately take the values into a DF
# There isn't really a good reason for two different methods, I was just tired on two different days
# I think the first process should be refactored using this method if possible
# Or the speed of the process for old vs refactored could be found at least
cDF = pd.DataFrame(completionList)
cDF.fillna("", inplace=True)

In [15]:
#so index is 0-26, row is a series with each element in the series being one column in order
#We dont need peak pay conditional as its own giant branch, can just have a flag, if peak pay peakFlag

for index1, row in cDF.iterrows():
    happyFlag = False
    stackFlag = False
    for index2, value in row.items():
        if("Customer Tips" in value):
            happyFlag = True
            ############################ PEAK PAY CONDITIONAL ##############################
            if("Peak Pay" in cDF[index2-1][index1]):
                basePay = cDF[index2-2][index1]
                peakPay = cDF[index2-1][index1]
                restName = cDF[index2+1][index1]
                if ("$" in restName):
                    tipPay = restName[restName.index("$"):]
                    restName = restName[:restName.index("$")]
                else:
                    tipPay = "$0.0"
                    restName = restName[:restName.index("-")]
                basePay = basePay[basePay.index("$"):]
                if("Total" in cDF[index2+2][index1]):
                    total = cDF[index2+2][index1]
                    total = total[total.index("$"):]
                else:
                    rest2 = cDF[index2+2][index1]
                    if("Total" in cDF[index2+3][index1]):
                        stackFlag = True
                        total = cDF[index2+3][index1]
                        total = total[total.index("$"):]
                    else:
                        stackFlag = False
                        print("ERROR 2")
            ################################## STANDARD CONDITIONAL ##################################
            else:
                basePay = cDF[index2-1][index1]
                restName = cDF[index2+1][index1]
                if ("$" in restName):
                    tipPay = restName[restName.index("$"):]
                    restName = restName[:restName.index("$")]
                else:
                    tipPay = "$0.0"
                basePay = basePay[basePay.index("$"):]
                if("Total" in cDF[index2+2][index1]):
                    total = cDF[index2+2][index1]
                    total = total[total.index("$"):]
                else:
                    rest2 = cDF[index2+2][index1]
                    if("Total" in cDF[index2+3][index1]):
                        stackFlag = True
                        total = cDF[index2+3][index1]
                        total = total[total.index("$"):]
                    else:
                        stackFlag = False
                        print("ERROR 2")
                    if(")" in rest2):
                        if not ("(" in rest2):
                            rest2 = ""
                            stackFlag = False               
    if not happyFlag:
            print("ERROR AT ROW:", index1)

ERROR AT ROW: 19


In [16]:
completedDF = pd.DataFrame(columns = ["Base Pay", "Tip", "Peak Pay", 
                                          "Restaurant Name", "Restaurant Name 2", "Tip 2", "Total"])
#index 1 is row #
#index 2 is col #
#happyFlag checks for the existence of Customer Tips in the row
# ----- but should be errorFlag and cover all problems
# ----- Cust Tips is important since we use it as an anchor point
for index1, row in cDF.iterrows():
    # FLAGS #
    happyFlag = False
    stackFlag = False
    # BASE CASE ELEMENT ATTRIBUTES
    basePay = 0
    tipPay = 0
    peakPay = 0
    totalPay = 0
    rest2 = "N/A"
    tip2 = "N/A"
    for index2, value in row.items():
        if("Customer Tips" in value):
            happyFlag = True
            ############################ PEAK PAY INDEXING ##############################
            if("Peak Pay" in cDF[index2-1][index1]):
                basePay = cDF[index2-2][index1]
                peakPay = cDF[index2-1][index1]
                if ("$" in peakPay):
                    peakPay = peakPay[peakPay.index("$"):] 
                else:
                    peakPay = "ERROR"
                restName = cDF[index2+1][index1]
                if ("$" in restName):
                    tipPay = restName[restName.index("$"):]
                    restName = restName[:restName.index("$")]
                else:
                    tipPay = "$0.00"
                    restName = restName[:restName.index("-")]
                basePay = basePay[basePay.index("$"):]
                if("Total" in cDF[index2+2][index1]):
                    total = cDF[index2+2][index1]
                    total = total[total.index("$"):]
                else:
                    rest2 = cDF[index2+2][index1]
                    if ("$" in rest2):
                        tip2 = rest2[rest2.index("$"):]
                        rest2 = rest2[:rest2.index("$")]
                    else:
                        tip2 = "$0.00"
                    if("Total" in cDF[index2+3][index1]):
                        stackFlag = True
                        total = cDF[index2+3][index1]
                        total = total[total.index("$"):]
                    else:
                        stackFlag = False
                        print("ERROR 2")
                    if(")" in rest2):
                        if not ("(" in rest2):
                            rest2 = "N/A"
                            tip2 = "N/A"
                            stackFlag = False            
            ################################## STANDARD INDEXING ##################################
            else:
                peakPay = "$0.00"
                basePay = cDF[index2-1][index1]
                restName = cDF[index2+1][index1]
                if ("$" in restName):
                    tipPay = restName[restName.index("$"):]
                    restName = restName[:restName.index("$")]
                else:
                    tipPay = "$0.00"
                basePay = basePay[basePay.index("$"):]
                if("Total" in cDF[index2+2][index1]):
                    total = cDF[index2+2][index1]
                    total = total[total.index("$"):]
                else:
                    rest2 = cDF[index2+2][index1]
                    if ("$" in rest2):
                        tip2 = rest2[rest2.index("$"):]
                        rest2 = rest2[:rest2.index("$")]
                    else:
                        tip2 = "$0.00"
                    if("Total" in cDF[index2+3][index1]):
                        stackFlag = True
                        total = cDF[index2+3][index1]
                        total = total[total.index("$"):]
                    else:
                        stackFlag = False
                        print("ERROR 2")
                    if(")" in rest2):
                        if not ("(" in rest2):
                            rest2 = "N/A"
                            tip2 = "N/A"
                            stackFlag = False
    if not happyFlag:
        newRowList = [0, 0, 0, 0, 0, 0, 0]
        completedDF.loc[len(completedDF)] = newRowList
    else:
        newRowList = [basePay, tipPay, peakPay, restName, rest2, tip2, total]
        completedDF.loc[len(completedDF)] = newRowList

In [17]:
offerDF["Offer File Name"] = offerFiles
offerDF = offerDF.join(timeDF)
offerDF = offerDF.rename(columns={"Time" : "Offer Timestamp"})
offerDF = offerDF.drop(["FileName"], axis=1)

/tmp/ipykernel_7900/2532294919.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offerDF["Offer File Name"] = offerFiles


In [18]:
completedDF["Completed File Name"] = compFiles
completedDF = completedDF.join(timeDF)
completedDF = completedDF.rename(columns={"Time" : "Completed Timestamp"})
completedDF = completedDF.drop(["FileName"], axis=1)

In [19]:
totalDF = completedDF.join(offerDF)
totalDF

,Base Pay,Tip,Peak Pay,Restaurant Name,Restaurant Name 2,Tip 2,Total,Completed File Name,Completed Timestamp,Dollars,Mileage,RestNames,Rest 2,Offer File Name,Offer Timestamp
0,$6.50,$7.00,$0.00,China Fusion,N/A,N/A,$13.50,IMG_2280.PNG,2023:06:24 13:17:06,13.5,5.5,China Fusion,,IMG_2279.PNG,2023:06:24 13:17:06
1,$4.00,$5.00,$0.00,Hooked on Sushi (Carlsbad),N/A,N/A,$9.00,IMG_2283.PNG,2023:06:24 13:38:54,9.0,3.8,Hooked on Sushi,,IMG_2281.PNG,2023:06:24 13:38:54
2,$5.50,$1.00,$0.00,Chick-fil-A (01507),N/A,N/A,$6.50,IMG_2288.PNG,2023:06:24 13:44:10,6.5,4.6,Chick-fil-A,,IMG_2284.PNG,2023:06:24 13:44:10
3,$11.50,$0.00,$0.00,7-Eleven (23302) -,Yukiya Sushi -,$0.00,$11.50,IMG_2296.PNG,2023:06:24 13:59:00,11.5,10.8,Sushi Yukiya,7-Eleven,IMG_2289.PNG,2023:06:24 13:59:00
4,$6.75,$0.00,$0.00,Starbucks (23194) -,N/A,N/A,$6.75,IMG_2301.PNG,2023:06:24 14:01:32,6.75,3.4,Starbucks,,IMG_2298.PNG,2023:06:24 14:01:32
5,$5.75,$5.00,$0.00,Pizza on tap,N/A,N/A,$10.75,IMG_2306.PNG,2023:06:24 14:28:18,10.75,4.2,Pizza on tap,,IMG_2303.PNG,2023:06:24 14:28:18
6,$5.00,$2.00,$0.00,Popeyes (1325),N/A,N/A,$7.00,IMG_2308.PNG,2023:06:24 14:32:10,7.0,3.7,Popeyes Louisiana Kitchen,,IMG_2304.PNG,2023:06:24 14:32:10
7,$11.25,$8.50,$0.00,Upper Crust Pizza,N/A,N/A,$19.75,IMG_2312.PNG,2023:06:24 15:09:25,19.75,10.5,Upper Crust Pizza,,IMG_2309.PNG,2023:06:24 15:09:25
8,$3.75,$1.00,$0.00,Wendy's (9076),N/A,N/A,$4.75,IMG_2320.PNG,2023:06:24 15:15:23,4.75,1.6,Wendy's,,IMG_2319.PNG,2023:06:24 15:15:23
9,$6.75,$0.00,$0.00,Advanced Auto Parts (via OneRail) (3752 _,N/A,N/A,$6.75,IMG_2322.PNG,2023:06:24 15:42:26,6.75,1.5,Advance Auto Parts,,IMG_2321.PNG,2023:06:24 15:42:26
